In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unidecode
from nltk import *

In [2]:
df = pd.read_csv("top10s.csv", encoding = "ISO-8859-1")

In [3]:
df.head()

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [4]:
df.isnull().sum()

Unnamed: 0    0
title         0
artist        0
top genre     0
year          0
bpm           0
nrgy          0
dnce          0
dB            0
live          0
val           0
dur           0
acous         0
spch          0
pop           0
dtype: int64

In [5]:
df = df[df['pop']>60]

In [6]:
df.rename(columns = {'top genre':'genre','pop':'popularity','nrgy':'energy','dnce':'dance','acous':'acoustic','spch':'speech','dur':'duration'}, inplace=True)

In [7]:
df.head()

,Unnamed: 0,title,artist,genre,year,bpm,energy,dance,dB,live,val,duration,acoustic,speech,popularity
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [8]:
df['genre'].unique()

array(['neo mellow', 'detroit hip hop', 'dance pop', 'pop',
       'canadian pop', 'hip pop', 'barbadian pop', 'atl hip hop',
       'australian pop', 'indie pop', 'british soul', 'chicago rap',
       'permanent wave', 'boy band', 'baroque pop', 'big room',
       'celtic rock', 'electro', 'complextro', 'art pop',
       'canadian hip hop', 'acoustic pop', 'candy pop', 'folk-pop',
       'metropopolis', 'house', 'australian hip hop', 'electropop',
       'colombian pop', 'canadian contemporary r&b',
       'irish singer-songwriter', 'tropical house', 'australian dance',
       'belgian edm', 'danish pop', 'hip hop', 'electronic trap', 'edm',
       'latin', 'electro house', 'brostep', 'contemporary country',
       'downtempo', 'escape room', 'alternative r&b'], dtype=object)

In [9]:
df['artist'] = df['artist'].apply(lambda x: x.lower())

In [10]:
df['artist'] = df['artist'].apply(lambda x:x.replace(" ",""))

In [11]:
df['genre'] = df['genre'].apply(lambda x:x.lower())

In [12]:
def remove_accents(s):
    s = unidecode.unidecode(s)
    return s
df['title'] = df['title'].apply(remove_accents)

In [13]:
df['title'] = df['title'].apply(lambda x:x.lower())

In [14]:
df['soup'] = df[['artist','genre']].apply(lambda x:' '.join(x), axis = 1)

In [15]:
 df.drop_duplicates(subset = 'title',inplace=True)

In [16]:
df = df[['Unnamed: 0','title','soup']]

In [17]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup']).toarray()

In [18]:
cos = cosine_similarity(count_matrix,count_matrix)

In [19]:
df = df.reset_index()
indices = pd.Series(df.index, index=df['title'])

In [20]:
def get_recommendations(title, cos=cos):
    idx = indices[title]
    cos_scores = list(enumerate(cos[idx]))
    cos_scores = sorted(cos_scores, key=lambda x: x[1], reverse=True)
    cos_scores = cos_scores[1:11]  
    song_indices = [i[0] for i in cos_scores]
    return df['title'].iloc[song_indices]

In [21]:
get_recommendations('tik tok')

19             your love is my drug
22                      take it off
32     blah blah blah (feat. 3oh!3)
51                    we r who we r
64                             blow
114                       die young
316                         praying
134                      #thatpower
3                       bad romance
6                          dynamite
Name: title, dtype: object